In [ ]:
from adler.dataclasses.AdlerPlanetoid import AdlerPlanetoid
from adler.science.PhaseCurve import PhaseCurve
from adler.dataclasses.AdlerData import AdlerData
import adler.utilities.science_utilities as utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u

In [ ]:
# ssObjectId of object to analyse
ssoid = "8268570668335894776"

# load object from local database
fname = "../tests/data/testing_database.db"
planetoid = AdlerPlanetoid.construct_from_SQL(ssoid, sql_filename=fname)

# retrieve observations in the r filter
obs_r = planetoid.observations_in_filter("r")

In [ ]:
# define the phase curve model using the SSObject data

sso_r = planetoid.SSObject_in_filter("r")
r_H = sso_r.H
r_G12 = sso_r.G12

pc = PhaseCurve(abs_mag=r_H * u.mag, phase_param=r_G12, model_name="HG12_Pen16")

In [ ]:
# calculate data - model residuals
res = obs_r.reduced_mag - pc.ReducedMag(obs_r.phaseAngle * u.degree).value

In [ ]:
# plot the observations with the SSObject phase curve
x_plot = "phaseAngle"
y_plot = "reduced_mag"

x = getattr(obs_r, x_plot)
y = getattr(obs_r, y_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, y, xerr, fmt="o")

# plot the phase curve model
alpha = np.linspace(0, np.amax(obs_r.phaseAngle)) * u.deg
red_mag = pc.ReducedMag(alpha)

# legend label for the phase curve model
pc_label = []
for x in pc.model_function.param_names:
    pc_label.append("{}={:.2f}".format(x, getattr(pc.model_function, x).value))
pc_label = ", ".join(pc_label)

ax1.plot(alpha.value, red_mag.value, c="k", label=pc_label)

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.legend()

plt.show()

In [ ]:
# plot the data - model residuals
x_plot = "phaseAngle"

x = getattr(obs_r, x_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, res, xerr, fmt="o")

ax1.axhline(0, c="k")

# indicate the standard deviations of the residuals
res_std = np.std(res)
for i in range(1, 4):
    ax1.axhline(res_std * i, ls=":", c="C{}".format(i), label="{} sigma".format(i))
    ax1.axhline(-res_std * i, ls=":", c="C{}".format(i))

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel("data - model")
ax1.legend()

plt.show()

In [ ]:
utils.sigma_clip(res, {"cenfunc": "median"})

In [ ]:
utils.sigma_clip(res)

In [ ]:
utils.outlier_std(res, res)

In [ ]:
df_obs_r = pd.DataFrame(obs_r.__dict__)
df_obs_r = df_obs_r.sort_values("midpointMjdTai")

In [ ]:
df_obs_r

In [ ]:
# step through all observations and check if photometry is outlying
for i in range(1, len(df_obs_r) + 1):
    #     print(df_obs_r.iloc[:i].index)
    _df = df_obs_r.iloc[:i]

    _res = _df["reduced_mag"] - pc.ReducedMag(np.array(_df["phaseAngle"]) * u.degree).value
    #     print(_res)

    #     print(utils.outlier_std(_res,res))
    print(utils.outlier_std(_res, _res))

In [ ]:
# step through all observations and check if new photometry is outlying

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

for i in range(2, len(df_obs_r) + 1):
    #     print(df_obs_r.iloc[:i].index)
    _df = df_obs_r.iloc[:i]

    _res = _df["reduced_mag"] - pc.ReducedMag(np.array(_df["phaseAngle"]) * u.degree).value
    _res = np.array(_res)

    #     print(_res)
    #     print(np.delete(_res,-1),[_res[-1]])

    outlier_flag = utils.outlier_std([_res[-1]], np.delete(_res, -1))
    print(outlier_flag, np.abs(_res[-1]), 3 * np.std(np.delete(_res, -1)))

    s1 = ax1.scatter(
        _df.iloc[-1]["phaseAngle"],
        _res[-1],
        c=_df.iloc[-1]["midpointMjdTai"],
        vmin=np.amin(df_obs_r["midpointMjdTai"]),
        vmax=np.amax(df_obs_r["midpointMjdTai"]),
    )

    if outlier_flag[0]:
        ax1.scatter(_df.iloc[-1]["phaseAngle"], _res[-1], c="r", marker="x")

ax1.axhline(0, c="k")
plt.colorbar(s1)
ax1.invert_yaxis()

plt.show()

In [ ]:
# step through all observations and check if new photometry is outlying

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

std_list = []
for i in range(2, len(df_obs_r) + 1):
    #     print(df_obs_r.iloc[:i].index)
    _df = df_obs_r.iloc[:i]

    _res = _df["reduced_mag"] - pc.ReducedMag(np.array(_df["phaseAngle"]) * u.degree).value
    _res = np.array(_res)

    #     print(_res)
    #     print(np.delete(_res,-1),[_res[-1]])

    outlier_flag = utils.outlier_std([_res[-1]], np.delete(_res, -1))
    print(outlier_flag, np.abs(_res[-1]), 3 * np.std(np.delete(_res, -1)))

    s1 = ax1.scatter(
        _df.iloc[-1]["midpointMjdTai"],
        _res[-1],
        c=_df.iloc[-1]["phaseAngle"],
        vmin=np.amin(df_obs_r["phaseAngle"]),
        vmax=np.amax(df_obs_r["phaseAngle"]),
    )

    if outlier_flag[0]:
        ax1.scatter(_df.iloc[-1]["midpointMjdTai"], _res[-1], c="r", marker="x")

    std_list.append(np.std(np.delete(_res, -1)))

std_list = np.array(std_list)

ax1.plot(np.array(df_obs_r["midpointMjdTai"])[1:], std_list * 3)

ax1.axhline(0, c="k")
plt.colorbar(s1)
ax1.invert_yaxis()

plt.show()

In [ ]:
std_list

In [ ]:
df_obs_r["midpointMjdTai"]

In [ ]:
# consider the residual compared to the uncertainty

In [ ]:
# plot the data - model residuals
x_plot = "phaseAngle"

x = getattr(obs_r, x_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, res, xerr, fmt="o")

ax1.axhline(0, c="k")

# indicate the standard deviations of the residuals
res_std = np.std(res)
for i in range(1, 4):
    ax1.axhline(res_std * i, ls=":", c="C{}".format(i), label="{} sigma".format(i))
    ax1.axhline(-res_std * i, ls=":", c="C{}".format(i))

std_err = 5
mask = np.abs(res) > (std_err * xerr)
ax1.scatter(x[mask], res[mask], c="r", marker="x", zorder=3)

mask2 = utils.outlier_sigma_diff(res, xerr, std_err)
ax1.scatter(x[mask], res[mask], edgecolor="r", facecolor="none", marker="o", zorder=3)

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel("data - model")
ax1.legend()

plt.show()

In [ ]:
# in general the residuals can be much larger than the photometric uncertainty

In [ ]:
res

In [ ]:
xerr